<div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 40px; margin-top: 0;">
    <div style="flex: 0 0 auto; margin-left: 0; margin-bottom: 0; margin-top: 0;">
        <img src="./pics/UCSD Logo.png" alt="UCSD Logo" style="width: 179px; margin-bottom: 0px; margin-top: 20px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/LANL-logo.png" alt="LANL Logo" style="width: 200px; margin-bottom: 0px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/prowess.png" alt="Prowess Logo" style="width: 200px; margin-bottom: 0px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/wildfire.png" alt="WildFire Logo" width="100"/>
    </div>
</div>

<h1 style="text-align: center; font-size: 48px; margin-top: 0;">Fire-Ready Forests Data Challenge</h1>

# Fast Fuels Demo

As a follow-up to the previous notebook, where you were introduced to FastFuels and a list was generated by calling its API, this notebook provides a brief demonstration of the FastFuels workflow and treelist generation. Specifically, this demo highlights the key steps involved in the FastFuels process, including:

1. Establishing a connection with TreeMap
2. Extracting a target raster from TreeMap
3. Generating a sample of trees from the target raster
4. Expanding the sample to represent the tree population within a specified Region of Interest (ROI)

For those interested in reviewing the underlying code, it can be found in the `utils` directory. You can also consult the [source repository](https://github.com/silvxlabs/fastfuels-core)

In [ ]:
# We start by importing our main libraries
import geopandas as gpd
from utils.treemap import TreeMapConnection
import matplotlib.pyplot as plt
import requests
import io
import zipfile
import os
import pandas as pd
import shutil

We will initiate by loading the region of interest, utilizing a shapefile that delineates the boundaries of Independence Lake. For the purpose of this notebook, we will forego recreating the polygons using plot centroids, a method employed in the previous notebook.

In [ ]:
# First, we load our region of interest
roi = gpd.read_file("https://wifire-data.sdsc.edu/data/ndp/Data_challenge_2024/shapefiles/independence_lake_boundary.geojson")

roi_utm = roi.to_crs(5070) # We transform to UTM Coordinate Reference System (CRS).

roi_utm.plot()

# TreeMap 2016: Tree-Level Forest Model

TreeMap 2016 provides a tree-level model of forests across the conterminous United States, aligning **FIA plot data** with a 30×30 meter raster grid. The dataset, developed using a random forests machine-learning algorithm, imputes FIA plot attributes based on LANDFIRE variables (forest cover, height, vegetation type, topography, climate, and disturbance history circa 2016). 

The main output is a GeoTIFF raster with imputed plot identifiers, linking to FIA databases for attributes such as forest type, basal area, and tree species. The dataset supports biomass estimation, snag hazard mapping, and fuel treatment planning. A separate snag hazard map is also included.

### Data Access
- Data Archive: https://www.fs.usda.gov/rds/archive/catalog/RDS-2021-0074
- TreeMap Explorer: https://apps.fs.usda.gov/lcms-viewer/treemap.html

### Citation
> Riley, K. L., Grenfell, I. C., Finney, M. A., & Shaw, J. D. (2021). *TreeMap 2016: A tree-level model of the forests of the conterminous United States circa 2016*. Fort Collins, CO: Forest Service Research Data Archive. [https://doi.org/10.2737/RDS-2021-0074](https://doi.org/10.2737/RDS-2021-0074).

Now that we have a better understanding of TreeMap, let's proceed with creating our treelist. To begin, we need to preprocess the zip file containing our data. Our first steps will be to:

1. Extract the contents of the zip file
2. Convert the extracted CSV file to a Parquet file for more efficient storage and analysis
3. Delete the original CSV file, as it will no longer be needed

In [ ]:
zip_url = "https://wifire-data.sdsc.edu/data/treemap/TreeMap2016_tree_table.zip"
extract_path = "./TreeMap2016"  # Temporary directory for extraction
parquet_output = "./TreeMap2016_tree_table.parquet"

os.makedirs(extract_path, exist_ok=True)

response = requests.get(zip_url)
with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
    zip_ref.extractall(extract_path)

# Read CSV and convert to Parquet
csv_path = os.path.join(extract_path, "TreeMap2016_tree_table.csv")
df = pd.read_csv(csv_path)
df.to_parquet(parquet_output, engine='pyarrow', index=False)

shutil.rmtree(extract_path)

Great, our tree table file is now ready! Next, we'll work with the TreeMap raster file. Since it's a large file (almost 10GB!), we'll use a remote connection to access it. 

In [ ]:
version="2016" # There is 2014 version of TreeMap, that's why we specify that we are working with 2016
seed=123

# First, we establish a connection with the TreeMap2016 
treemap_connection = TreeMapConnection(
    treemap_path=f"https://wifire-data.sdsc.edu/data/treemap/TreeMap{version}.tif", # This is the remote URL
    tree_table_path=f"./TreeMap{version}_tree_table.parquet",
    version=version,
)

# Now we extract our target raster, based on our Region of Interest
treemap_raster_extraction = treemap_connection.extract_window(
    roi_utm,
    projection_padding_meters=15 * treemap_connection.raster_resolution,
    interpolation_padding_cells=4,
)

# We extract the plots corresponding to that raster
treemap_plots = treemap_connection.get_plots_dataframe_from_raster(
    treemap_raster_extraction
)

# We generate a sample from the representative plots of our Region of Interest
tree_sample = treemap_connection.query_trees_by_plots(treemap_plots)

Now that we have generated a sample, let's examine it.

In [ ]:
tree_sample

In [ ]:
# Create a histogram for HT (Tree Height)
plt.figure(figsize=(8, 5))
plt.hist(tree_sample['HT'], bins=30, edgecolor='black', alpha=0.7)
plt.xlabel("Tree Height (HT)")
plt.ylabel("Frequency")
plt.title("Distribution of Tree Height (HT) - Sample")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Create a histogram for DIA (Tree Diameter)
plt.figure(figsize=(8, 5))
plt.hist(tree_sample['DIA'], bins=30, edgecolor='black', alpha=0.7)
plt.xlabel("Tree Diameter (DIA)")
plt.ylabel("Frequency")
plt.title("Distribution of Tree Diameter (DIA) - Sample")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

Now we can proceed to extend our sample to our full region of interest.

In [ ]:
tree_population = tree_sample.expand_to_roi(
    "inhomogeneous_poisson",
    roi_utm,
    plots=treemap_plots,
    intensity_resolution=15,
    seed=seed,
)
tree_population

We will create the same histograms we created for the sample.

In [ ]:
# Histogram for HT (Tree Height)
plt.figure(figsize=(8, 5))
plt.hist(tree_population['HT'], bins=30, edgecolor='black', alpha=0.7)
plt.xlabel("Tree Height (HT)")
plt.ylabel("Frequency")
plt.title("Distribution of Tree Height (HT) - Population")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Create a histogram for DIA (Tree Diameter)
plt.figure(figsize=(8, 5))
plt.hist(tree_population['DIA'], bins=30, edgecolor='black', alpha=0.7)
plt.xlabel("Tree Diameter (DIA)")
plt.ylabel("Frequency")
plt.title("Distribution of Tree Diameter (DIA) - Population")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

### Acknowledgment

This Jupyter Notebook utilizes core code developed by Anthony Marcozzi from the New Mexico Consortium. The content, including analysis and visualizations, builds upon Anthony's foundational work, and we appreciate his contribution to the underlying codebase. Proper credit is appreciated if this notebook is shared or adapted for further use.